In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from utils import get_imagenet_classes

In [2]:
# create mask to correspond between imagenet-a labels and imagenet-1k

all_wnids, imagenet_a_wnids, imagenet_a_mask = get_imagenet_classes()

In [4]:
# drive mount / link to directory

val_dir1 = "C:/Users/laure/Documents/nat_advs_proj/imagenet-a-split/test"

#load val/test to keras dataset from directory provided
# the image labels are stored as integers according to the folder names and the 
# ordering provided by imagenet_a_wnids
batch_size = 32
val_dataset1 = tf.keras.preprocessing.image_dataset_from_directory(val_dir1, labels='inferred',batch_size=None, label_mode="categorical",
                                                                    shuffle=False, class_names = imagenet_a_wnids)
image_labels = np.array([y for x, y in val_dataset1])


# preprocessing randomly crops to 224x224 and standard ResNet processing is applied
def preproc(tensor,y):
    tensor = tf.image.random_crop(value=tensor, size=(224,224, 3))
    tensor = preprocess_input(tensor)
    return tensor, y

normalized_ds = val_dataset1.map(preproc).batch(32)

#load resnet50 with imagenet-1k weights
base_model = ResNet50(include_top=True,
             weights='imagenet',
             input_tensor=None,
             input_shape=None,
             pooling='avg')


Found 917 files belonging to 200 classes.


In [6]:
def get_predictions(dset, net=None, mask=None):

    # predict labels based on network
    outputs = net.predict(dset)

    # mask outputs to only be imagenet-a related
    mask_outputs = outputs[:,mask]
    
    # take argmax of the imagenet-a related labels ONLY
    pred = np.argmax(mask_outputs,axis=1)

    # compare to real labels
    num_correct = np.array(pred==image_labels).sum()

    #output correct examples
    correct = np.argwhere(pred==image_labels)
    
    return correct, num_correct


def get_imagenet_a_results(loader, net, mask):
    correct, num_correct = get_predictions(loader, net, mask)
    acc = num_correct / image_labels.shape[0]
    print('Accuracy (%):', round(100*acc, 4))

    return correct

In [7]:
correct_labs = get_imagenet_a_results(normalized_ds, base_model, imagenet_a_mask)

29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step
Accuracy (%): 5.8888


In [8]:
# list of correctly predicted examples
correct_labs

array([[  7],
       [122],
       [230],
       [259],
       [281],
       [302],
       [334],
       [351],
       [420],
       [479],
       [511],
       [520],
       [528],
       [536],
       [542],
       [549],
       [557],
       [558],
       [566],
       [588],
       [597],
       [599],
       [612],
       [613],
       [627],
       [646],
       [649],
       [650],
       [654],
       [669],
       [676],
       [677],
       [686],
       [701],
       [717],
       [718],
       [722],
       [731],
       [753],
       [755],
       [765],
       [774],
       [784],
       [786],
       [827],
       [843],
       [850],
       [851],
       [877],
       [882],
       [894],
       [906],
       [907],
       [912]], dtype=int64)